In [1]:
import pandas as pd
pat_id_offset = pd.read_csv('../data/intermediate/id_offset.csv', header=None)
test_lab = pd.read_csv('../data/intermediate/test_lab_data.csv', header=None)

In [2]:
columns = ['patientunitstayid', 'labname', 'labresult', 'labresultoffset']
test_lab.columns=columns
pat_id_offset.columns=['patientunitstayid', 'earlier_time']
pat_id_offset.set_index('patientunitstayid', inplace=True)

In [3]:
test_lab.set_index(['patientunitstayid','labresultoffset', 'labname'], inplace=True)
test_lab = test_lab.unstack(level=2)

In [4]:
test_lab.columns = test_lab.columns.droplevel()

In [5]:
test_lab.reset_index(level=1, inplace=True)

In [6]:
test_lab = test_lab.merge(pat_id_offset, left_index=True, right_index=True)

In [7]:
test_lab['time_to_detect'] = test_lab['earlier_time'] - test_lab['labresultoffset']

In [8]:
pd.option_context("mode.chained_assignment", None)
test_lab = test_lab[test_lab['time_to_detect']/60>=12]
test_lab.loc[test_lab['time_to_detect']/60<=24, 'positive_window'] = 1

In [9]:
test_lab.fillna(0, inplace=True)

In [10]:
test_lab.reset_index(inplace=True)
test_lab['time_to_detect'] = pd.to_timedelta(test_lab['time_to_detect'], unit='minute')

In [11]:
import sys, os
sys.path.insert(0, os.path.abspath('../src/'))
import lab_wrangling
import importlib

In [12]:
columns = ['creatinine', 'BUN', 'WBC x 1000']
funcs = ['min', 'median', 'max', 'var']
test_lab.reset_index(inplace=True)
df = test_lab
test_lab = lab_wrangling.calc_stats_windows(df, columns, funcs)

In [1]:
test_lab

In [14]:
print(len(test_lab))
test_lab.dropna(inplace=True)
print(len(test_lab))

7
3


In [2]:
test_lab

In [58]:
def calc_stats_windows(df, columns, funcs):
    df['time_to_detect'] = pd.to_timedelta(test_lab['time_to_detect'], unit='minute')
    df.set_index('time_to_detect', inplace=True)
    sum_df = df.groupby('patientunitstayid')[columns].apply(lambda x: x.rolling('2d', min_periods=2).agg(funcs)) 
    sum_df.columns = sum_df.columns.map('_'.join)
    columns = sum_df.columns.tolist()
    columns.append('patientunitstayid')
    sum_df = pd.concat([sum_df, df['patientunitstayid']], axis=1, ignore_index=True)
    sum_df.columns = columns
    sum_df.reset_index(inplace=True)
    sum_df.set_index(['patientunitstayid', 'time_to_detect'], inplace=True)
    df.reset_index(inplace=True)
    df.set_index(['patientunitstayid', 'time_to_detect'], inplace=True)
    df = pd.concat([df, sum_df], axis=1, ignore_index=False)
    return df

In [3]:
test_lab